***NOTE: This notebook has been adapted from https://stackabuse.com/text-classification-with-python-and-scikit-learn/ for a tutorial in NLP from the UCL ICH coding club***

# Classification of reviews

The dataset that we are going to use for this article can be downloaded from: http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz

The dataset consists of a total of 2000 documents. Half of the documents contain positive reviews regarding a movie while the remaining half contains negative reviews. 

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/ferran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
movie_data = load_files(r"txt_sentoken")
X, y = movie_data.data, movie_data.target
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Unique Labels:", unique_elements)
print("Proportions:", counts_elements*100/len(y), "%")
print("Number of reviews:", len(y))
print(y[0:2])
print(X[0:2])

Unique Labels: [0 1]
Proportions: [50. 50.] %
Number of reviews: 2000
[0 1]
[b"arnold schwarzenegger has been an icon for action enthusiasts , since the late 80's , but lately his films have been very sloppy and the one-liners are getting worse . \nit's hard seeing arnold as mr . freeze in batman and robin , especially when he says tons of ice jokes , but hey he got 15 million , what's it matter to him ? \nonce again arnold has signed to do another expensive blockbuster , that can't compare with the likes of the terminator series , true lies and even eraser . \nin this so called dark thriller , the devil ( gabriel byrne ) has come upon earth , to impregnate a woman ( robin tunney ) which happens every 1000 years , and basically destroy the world , but apparently god has chosen one man , and that one man is jericho cane ( arnold himself ) . \nwith the help of a trusty sidekick ( kevin pollack ) , they will stop at nothing to let the devil take over the world ! \nparts of this are actual

# Text Preprocessing
Once the dataset has been imported, the next step is to preprocess the text. Text may contain numbers, special characters, and unwanted spaces. 




In [3]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/ferran/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
documents = []

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [lemmatizer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)


In [5]:
type(documents)

list

In [6]:
documents_df = pd.DataFrame(data=documents, columns=["text"])
documents_df.head()

,text
0,arnold schwarzenegger ha been an icon for acti...
1,good film are hard to find these day ngreat fi...
2,quaid star a man who ha taken up the proffesio...
3,we could paraphrase michelle pfieffer characte...
4,kolya is one of the richest film ve seen in so...


# Vectorization through Bag-of-Words

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [8]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 1, 0, 0]])

In [9]:
X.shape

(2000, 1500)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
print("Length of training set: ", X_train.shape[0])
print("Length of test set: ", X_test.shape[0])

Length of training set:  1600
Length of test set:  400


# Training Text Classification Model and Predicting Sentiment

Loading machine learning model-> random forest

In [12]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)

![](randomforest.jpg)

Fit the model using the training data: learn relationship between X (feature matrix) and Y (postivie/negative reviews)

In [13]:
classifier.fit(X_train, y_train) 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

### Make predictions on the test set

In [14]:
y_pred = classifier.predict(X_test)

# Evaluating the Model

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[167  41]
 [ 19 173]]
              precision    recall  f1-score   support

           0       0.90      0.80      0.85       208
           1       0.81      0.90      0.85       192

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400

0.85


# Try with some sentences

In [16]:
mysentences = ["This is an awful movie","This is an incredible movie. The best one I have ever seen. I would definetly recommend it to everyone"]

In [17]:
documents2 = []

for sen in range(0, len(mysentences)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(mysentences[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [lemmatizer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents2.append(document)

In [18]:
documents2

['this is an awful movie',
 'this is an incredible movie the best one have ever seen would definetly recommend it to everyone']

In [19]:
X2 = vectorizer.transform(documents2).toarray()

In [20]:
classifier.predict(X2)

array([0, 0])

In [21]:
classifier.predict_proba(X2)

array([[0.757, 0.243],
       [0.626, 0.374]])

### Common case in ML...
### Why?

# 2. Spacy

In [31]:
import spacy 
from spacy import displacy

In [32]:
nlp = spacy.load("en_core_web_sm")

In [35]:
text = "Aspirin treats headaches. Aspirin can help prevent heart attacks"

In [36]:
doc = nlp(text)

In [37]:
doc[0]

Aspirin

In [40]:
doc[0].vector

array([-1.5145166 ,  1.0732374 ,  0.5517379 , -0.01871049,  2.6456032 ,
       -0.18174338, -2.6634395 ,  5.2556896 , -0.33252132,  0.27583006,
        2.5330546 , -0.059816  ,  1.093798  , -1.1808696 , -1.1488907 ,
        4.0781755 ,  1.358101  ,  0.4007492 , -1.902514  , -0.20487535,
       -1.3795193 , -0.41477293,  1.7291825 ,  0.7380515 , -1.3701922 ,
       -2.2865162 , -2.0949988 , -0.6220954 , -1.6144851 ,  0.12104107,
        1.9048204 ,  3.541529  , -0.44125214, -0.7613086 , -0.12625363,
       -1.2551541 ,  0.32477027,  0.91225255, -2.566049  ,  0.54549205,
        2.3800344 ,  2.371096  , -3.9211543 , -2.6386678 ,  0.03198814,
        0.23681575, -1.5880487 , -2.5741482 , -0.72786427,  0.56942713,
        2.242044  , -1.3190459 , -0.9748065 ,  0.1570418 , -3.35715   ,
        1.2155153 ,  2.5696006 , -1.6757537 ,  2.7926946 ,  0.3263369 ,
       -0.3917737 , -0.8415663 , -3.2144    ,  0.92596483, -0.334371  ,
        0.69201636,  3.10426   , -1.5518475 , -1.0355687 , -1.24

In [42]:
displacy.render(doc, style = "dep")